# RR tournament scheduling tests

## imports

In [1]:
!pip install pyomo

In [2]:
!apt-get install -y -qq glpk-utils
!apt-get install -y -qq coinor-cbc
!pip install gurobipy

In [3]:
import pyomo
from pyomo.environ import ConcreteModel, RangeSet, Var, Binary, Constraint, Objective, \
  NonNegativeIntegers, Integers, SolverFactory, minimize, ConstraintList, Param, Any, Set, NonNegativeReals
import numpy as np
import time

from itertools import combinations
import os
import json
import argparse
import sys

## problem def

### single solve optimization

#### without symmetry breaking constr

In [4]:
def solve(n, solvers=['glpk']):
    model = ConcreteModel()

    model.W = RangeSet(0, (n-1)-1)
    model.P = RangeSet(0, (n//2)-1)
    model.I = RangeSet(0, n-1)
    model.J = RangeSet(0, n-1)
    # binary decision var
    model.X = Var(model.W, model.P, model.I, model.J, domain=Binary)
    # auxiliary vars for obj func
    model.home = Var(model.I, domain=NonNegativeIntegers)
    model.away = Var(model.I, domain=NonNegativeIntegers)
    model.diff = Var(model.I, domain=NonNegativeReals)

    # rules definition
    def one_match_rule(model, i):
        return (
            sum(model.X[w, p, i, j] for w in model.W for p in model.P for j in model.J) +
            sum(model.X[w, p, j, i] for w in model.W for p in model.P for j in model.J)
            == n - 1
        )
    def no_self_match_rule(model, i):
        return sum(model.X[w, p, i, i] for w in model.W for p in model.P) == 0

    def symmetry_rule(model, i, j):
        if i < j:
            return (
                sum(model.X[w, p, i, j] for w in model.W for p in model.P) +
                sum(model.X[w, p, j, i] for w in model.W for p in model.P)
                == 1
            )
        else:
            return Constraint.Skip

    def one_match_per_week_rule(model, w, i):
        return sum(model.X[w, p, i, j] + model.X[w, p, j, i] for p in model.P for j in model.J if j != i) <= 1

    def max_one_per_period_per_week_rule(model, w, i, j):
        return sum(model.X[w, p, i, j] for p in model.P) <= 1

    def max_one_game_per_match_rule(model, i, j):
        return sum(model.X[w, p, i, j] for w in model.W for p in model.P) <= 1

    def max_two_matches_per_period_rule(model, i, p):
        return (
            sum(model.X[w, p, i, j] for w in model.W for j in model.J) +
            sum(model.X[w, p, j, i] for w in model.W for j in model.J)
            <= 2
        )

    model.one_match_per_team = Constraint(model.I, rule=one_match_rule)
    model.no_self_match = Constraint(model.I, rule=no_self_match_rule)
    model.symmetry = Constraint(model.I, model.J, rule=symmetry_rule)
    model.one_match_per_week = Constraint(model.W, model.I, rule=one_match_per_week_rule)
    model.max_one_per_period_per_week = Constraint(model.W, model.I, model.J, rule=max_one_per_period_per_week_rule)
    model.max_one_game_per_match = Constraint(model.I, model.J, rule=max_one_game_per_match_rule)
    model.max_two_matches_per_period = Constraint(model.I, model.P, rule=max_two_matches_per_period_rule)

    # auxiliary constraints for obj function
    def home_games_rule(model, i):
        return model.home[i] == sum(model.X[w, p, i, j] for w in model.W for p in model.P for j in model.J if i != j)
    def away_games_rule(model, i):
        return model.away[i] == sum(model.X[w, p, j, i] for w in model.W for p in model.P for j in model.J if i != j)
    model.home_games = Constraint(model.I, rule=home_games_rule)
    model.away_games = Constraint(model.I, rule=away_games_rule)

    # additional constraints for efficiency
    def tot_matches_rule(model):
        return sum(model.X[w,p,i,j] for w in model.W for p in model.P for i in model.I for j in model.J) == n*(n - 1)//2
    model.tot_matches = Constraint(rule=tot_matches_rule)

    model.z = Var(domain=NonNegativeReals)
    model.balance_max = ConstraintList()
    for i in model.I:
        model.balance_max.add(model.home[i] - model.away[i] <= model.z)
        model.balance_max.add(model.away[i] - model.home[i] <= model.z)
    model.obj = Objective(expr=model.z, sense=minimize)

    solver = SolverFactory('glpk')
    result = solver.solve(model, tee=False, timelimit=300)

    solution = np.zeros((n-1, n//2, n, n))
    for i in model.X.get_values().keys():
        if model.X[i].value is not None and abs(model.X[i].value) > 1e-6:
            solution[i[0], i[1], i[2], i[3]] = 1
    return result, solution

#### with symmetry breaking constr

In [5]:
def solveSB(n, solvers=['glpk']):
    model = ConcreteModel()

    model.W = RangeSet(0, (n-1)-1)
    model.P = RangeSet(0, (n//2)-1)
    model.I = RangeSet(0, n-1)
    model.J = RangeSet(0, n-1)
    model.X = Var(model.W, model.P, model.I, model.J, domain=Binary)
    model.home = Var(model.I, domain=NonNegativeIntegers)
    model.away = Var(model.I, domain=NonNegativeIntegers)
    model.diff = Var(model.I, domain=NonNegativeReals)

    # rules definition
    def one_match_rule(model, i):
        return (
            sum(model.X[w, p, i, j] for w in model.W for p in model.P for j in model.J) +
            sum(model.X[w, p, j, i] for w in model.W for p in model.P for j in model.J)
            == n - 1
        )
    def no_self_match_rule(model, i):
        return sum(model.X[w, p, i, i] for w in model.W for p in model.P) == 0
    def symmetry_rule(model, i, j):
        if i < j:
            return (
                sum(model.X[w, p, i, j] for w in model.W for p in model.P) +
                sum(model.X[w, p, j, i] for w in model.W for p in model.P)
                == 1
            )
        else:
            return Constraint.Skip
    def one_match_per_week_rule(model, w, i):
        return sum(model.X[w, p, i, j] + model.X[w, p, j, i] for p in model.P for j in model.J if j != i) <= 1
    def max_one_per_period_per_week_rule(model, w, i, j):
        return sum(model.X[w, p, i, j] for p in model.P) <= 1
    def max_one_game_per_match_rule(model, i, j):
        return sum(model.X[w, p, i, j] for w in model.W for p in model.P) <= 1
    def max_two_matches_per_period_rule(model, i, p):
        return (
            sum(model.X[w, p, i, j] for w in model.W for j in model.J) +
            sum(model.X[w, p, j, i] for w in model.W for j in model.J)
            <= 2
        )
    model.one_match_per_team = Constraint(model.I, rule=one_match_rule)
    model.no_self_match = Constraint(model.I, rule=no_self_match_rule)
    model.symmetry = Constraint(model.I, model.J, rule=symmetry_rule)
    model.one_match_per_week = Constraint(model.W, model.I, rule=one_match_per_week_rule)
    model.max_one_per_period_per_week = Constraint(model.W, model.I, model.J, rule=max_one_per_period_per_week_rule)
    model.max_one_game_per_match = Constraint(model.I, model.J, rule=max_one_game_per_match_rule)
    model.max_two_matches_per_period = Constraint(model.I, model.P, rule=max_two_matches_per_period_rule)

    # auxiliary constraints for obj function
    def home_games_rule(model, i):
        return model.home[i] == sum(model.X[w, p, i, j] for w in model.W for p in model.P for j in model.J if i != j)
    def away_games_rule(model, i):
        return model.away[i] == sum(model.X[w, p, j, i] for w in model.W for p in model.P for j in model.J if i != j)
    def diff_lower_bound1(model, i):
        return model.diff[i] >= model.home[i] - model.away[i]
    def diff_lower_bound2(model, i):
        return model.diff[i] >= model.away[i] - model.home[i]
    model.home_games = Constraint(model.I, rule=home_games_rule)
    model.away_games = Constraint(model.I, rule=away_games_rule)
    model.diff_lb1 = Constraint(model.I, rule=diff_lower_bound1)
    model.diff_lb2 = Constraint(model.I, rule=diff_lower_bound2)

    # additional constraints for efficiency
    def tot_matches_rule(model):
        return sum(model.X[w,p,i,j] for w in model.W for p in model.P for i in model.I for j in model.J) == n*(n - 1)//2
    model.tot_matches = Constraint(rule=tot_matches_rule)

    # symmetry breaking
    def fix_first_week_rule(model, p):
        i = 2 * p
        j = 2 * p + 1
        return model.X[0, p, i, j] == 1  # Fix home/away arbitrarily
    def fix_team0_schedule_rule(model, w):
        j = w + 1
        return sum(model.X[w, p, 0, j] + model.X[w, p, j, 0] for p in model.P) == 1
    model.fix_first_week = Constraint(model.P, rule=fix_first_week_rule)
    model.team0_schedule = Constraint(model.W, rule=fix_team0_schedule_rule)

    # objective function
    model.z = Var(domain=NonNegativeReals)
    model.balance_max = ConstraintList()
    for i in model.I:
        model.balance_max.add(model.home[i] - model.away[i] <= model.z)
        model.balance_max.add(model.away[i] - model.home[i] <= model.z)
    model.obj = Objective(expr=model.z, sense=minimize)

    solver = SolverFactory('glpk')
    result = solver.solve(model, tee=False, timelimit=300)

    solution = np.zeros((n-1, n//2, n, n))
    for i in model.X.get_values().keys():
        if model.X[i].value is not None and abs(model.X[i].value) > 1e-6:
            solution[i[0], i[1], i[2], i[3]] = 1
    return result, solution

### double solve decision
no optimization possible as home/away is not tracked to halve var dim

In [6]:
def solve1(n, solvers):
    # first model
    model = ConcreteModel()
    model.W = RangeSet(0, (n-1)-1)
    model.I = RangeSet(0, n-1)
    model.J = RangeSet(0, n-1)
    model.M = Var(model.W, model.I, model.J, domain=Binary)

    # rules definition
    def one_match_per_week_per_team_rule(model, w, i):
        return sum(model.M[w, i, j] for j in model.J if i < j) + sum(model.M[w, j, i] for j in model.J if j < i) == 1
    def one_match_pair_rule(model, i, j):
        if i < j:
            return sum(model.M[w, i, j] for w in model.W) == 1
        return Constraint.Skip
    # def matches_per_week_rule(model, w):
    #     return sum(model.M[w, i, j] for i in model.I for j in model.J if i < j) == n // 2
    model.one_match_per_week_per_team = Constraint(model.W, model.I, rule=one_match_per_week_per_team_rule)
    model.one_match_pair = Constraint(model.I, model.J, rule=one_match_pair_rule)
    # model.matches_per_week = Constraint(model.W, rule=matches_per_week_rule)

    # objective function
    model.obj = Objective(expr=1, sense=minimize)

    solver = SolverFactory(solvers[0])
    result = solver.solve(model, tee=False, timelimit=150)

    # extract solution matrix
    model1_sol = np.zeros((n-1, n, n))
    for i in model.M.get_values().keys():
        if model.M[i].value is not None and abs(model.M[i].value) > 1e-6:
            model1_sol[i[0], i[1], i[2]] = 1

    # second model
    model2 = ConcreteModel()
    model2.W = RangeSet(0, (n-1)-1)
    model2.P = RangeSet(0, (n//2)-1)
    model2.I = RangeSet(0, n-1)
    model2.J = RangeSet(0, n-1)
    model2.X = Var(model2.W, model2.P, model2.I, model2.J, domain=Binary)
    model2.M = Var(model2.W, model2.I, model2.J, domain=Binary)

    # initialize model2 from model1
    for w in model2.W:
        for i in model2.I:
            for j in model2.J:
                if i < j:
                    model2.M[w, i, j].fix(model1_sol[w, i, j])

    def assignment_rule(model, w, p, i, j):
        return model2.X[w, p, i, j] <= model.M[w, i, j]
    model2.assignmentConstraint = Constraint(model2.W, model2.P, model2.I, model2.J, rule=assignment_rule)

    def one_match_per_period_rule(model, w,i,j):
        if model1_sol[w,i,j] == 1:
            return sum(model2.X[w, p, i, j] for p in model2.P) == 1
        return Constraint.Skip
    def period_per_week_rule(model, w, p):
        return sum(model2.X[w, p, i, j] for i in model2.I for j in model2.J) <= 1
    def max_period_per_team_rule(model, p, i):
        return sum(model2.X[w, p, i, j] for w in model2.W for j in model2.J if i < j) + sum(model2.X[w, p, j, i] for w in model2.W for j in model2.J if j < i) <= 2
    model2.one_match_per_period = Constraint(model2.W, model2.I, model2.J, rule=one_match_per_period_rule)
    model2.period_per_week = Constraint(model2.W, model2.P, rule=period_per_week_rule)
    model2.max_period_per_team = Constraint(model2.P, model2.I, rule=max_period_per_team_rule)

    # objective function
    model2.obj = Objective(expr=1, sense=minimize)

    solver2 = SolverFactory(solvers[1])
    result2 = solver2.solve(model2, tee=False, timelimit=150)

    solution2 = np.zeros((n-1, n//2, n, n))
    for i in model2.X.get_values().keys():
        if model2.X[i].value is not None and abs(model2.X[i].value) > 1e-6:
            solution2[i[0], i[1], i[2], i[3]] = 1

    return result, model1_sol, result2, solution2

### double solve optimization

In [7]:
def solve2(n, solvers=['glpk', 'glpk']):
    # first model
    model = ConcreteModel()
    model.W = RangeSet(0, (n-1)-1)
    model.I = RangeSet(0, n-1)
    model.J = RangeSet(0, n-1)
    # binary decision var
    model.M = Var(model.W, model.I, model.J, domain=Binary)
    # auxiliary vars for obj func
    model.Home = Var(model.I, domain=NonNegativeIntegers)
    model.Away = Var(model.I, domain=NonNegativeIntegers)
    model.Z = Var(domain=NonNegativeIntegers)

    # rules definition
    def one_match_per_week_per_team_rule(model, w, i):
        return sum(model.M[w, i, j] for j in model.J if i != j) + sum(model.M[w, j, i] for j in model.J if j != i) == 1
    def one_match_pair_rule(model, i, j):
        if not i == j:
            return sum(model.M[w, i, j] for w in model.W) + sum(model.M[w, j, i] for w in model.W) == 1
        return Constraint.Skip
    # def matches_per_week_rule(model, w):
    #     return sum(model.M[w, i, j] for i in model.I for j in model.J if i < j) == n // 2
    model.one_match_per_week_per_team = Constraint(model.W, model.I, rule=one_match_per_week_per_team_rule)
    model.one_match_pair = Constraint(model.I, model.J, rule=one_match_pair_rule)
    # model.matches_per_week = Constraint(model.W, rule=matches_per_week_rule)

    # auxiliary constraints for obj function
    def home_games_rule(model, i):
        return model.Home[i] == sum(model.M[w, i, j] for w in model.W for j in model.J if i != j)
    def away_games_rule(model, i):
        return model.Away[i] == sum(model.M[w, j, i] for w in model.W for j in model.J if i != j)
    model.home_games = Constraint(model.I, rule=home_games_rule)
    model.away_games = Constraint(model.I, rule=away_games_rule)

    # objective function
    model.balance_max = ConstraintList()
    for i in model.I:
        model.balance_max.add(model.Home[i] - model.Away[i] <= model.Z)
        model.balance_max.add(model.Away[i] - model.Home[i] <= model.Z)
    model.obj = Objective(expr=model.Z, sense=minimize)

    solver = SolverFactory(solvers[0])
    result = solver.solve(model, tee=False, timelimit=200)

    # extract solution matrix
    model1_sol = np.zeros((n-1, n, n))
    for i in model.M.get_values().keys():
        if model.M[i].value is not None and abs(model.M[i].value) > 1e-6:
            model1_sol[i[0], i[1], i[2]] = 1

    # second model
    model2 = ConcreteModel()
    model2.W = RangeSet(0, (n-1)-1)
    model2.P = RangeSet(0, (n//2)-1)
    model2.I = RangeSet(0, n-1)
    model2.J = RangeSet(0, n-1)
    model2.X = Var(model2.W, model2.P, model2.I, model2.J, domain=Binary)
    model2.M = Var(model2.W, model2.I, model2.J, domain=Binary)
    model2.Home = Var(model2.I, domain=NonNegativeIntegers)
    model2.Away = Var(model2.I, domain=NonNegativeIntegers)
    model2.Z = Var(domain=NonNegativeReals)

    # initialize model2 from model1
    for w in model2.W:
        for i in model2.I:
            for j in model2.J:
                if i < j:
                    model2.M[w, i, j].fix(model1_sol[w, i, j])

    def assignment_rule(model, w, p, i, j):
        return model2.X[w, p, i, j] <= model.M[w, i, j]
    model2.assignmentConstraint = Constraint(model2.W, model2.P, model2.I, model2.J, rule=assignment_rule)

    def one_match_per_period_rule(model, w,i,j):
        if model1_sol[w,i,j] == 1:
            return sum(model2.X[w, p, i, j] for p in model2.P) == 1
        return Constraint.Skip
    def period_per_week_rule(model, w, p):
        return sum(model2.X[w, p, i, j] for i in model2.I for j in model2.J if i != j) <= 1
    def max_period_per_team_rule(model, p, i):
        return sum(model2.X[w, p, i, j] for w in model2.W for j in model2.J if i != j) + sum(model2.X[w, p, j, i] for w in model2.W for j in model2.J if i != j) <= 2
    model2.one_match_per_period = Constraint(model2.W, model2.I, model2.J, rule=one_match_per_period_rule)
    model2.period_per_week = Constraint(model2.W, model2.P, rule=period_per_week_rule)
    model2.max_period_per_team = Constraint(model2.P, model2.I, rule=max_period_per_team_rule)

    # auxiliary constraints for obj function
    def home_games_rule(model, i):
        return model2.Home[i] == sum(model2.X[w, p, i, j] for w in model2.W for p in model2.P for j in model2.J if i != j)
    def away_games_rule(model, i):
        return model2.Away[i] == sum(model2.X[w, p, j, i] for w in model2.W for p in model2.P for j in model2.J if i != j)
    model2.home_games = Constraint(model2.I, rule=home_games_rule)
    model2.away_games = Constraint(model2.I, rule=away_games_rule)

    # objective function
    model2.balance_max = ConstraintList()
    for i in model.I:
        model2.balance_max.add(model2.Home[i] - model2.Away[i] <= model2.Z)
        model2.balance_max.add(model2.Away[i] - model2.Home[i] <= model2.Z)
    model2.obj = Objective(expr=model2.Z, sense=minimize)

    solver2 = SolverFactory(solvers[1])
    result2 = solver2.solve(model2, tee=False, timelimit=100)

    solution2 = np.zeros((n-1, n//2, n, n))
    for i in model2.X.get_values().keys():
        if model2.X[i].value is not None and abs(model2.X[i].value) > 1e-6:
            solution2[i[0], i[1], i[2], i[3]] = 1

    return result, model1_sol, result2, solution2

### 3 var solve decision

In [8]:
def solve3dec(n, solvers=['glpk']):
    model = ConcreteModel()

    model.W = RangeSet(0, (n-1)-1)
    model.P = RangeSet(0, (n//2)-1)
    model.I = RangeSet(0, n-1)
    model.J = RangeSet(0, n-1)
    model.M = RangeSet(0, (n*(n - 1)//2)-1)
    model.X = Var(model.W, model.M, domain=Binary)  # match m in week w
    model.H = Var(model.M, domain=Binary)  # match m is home for i
    model.Q = Var(model.P, model.I, model.J, domain=Binary)  # period of match m

    l = [(i,j) for i in range(n) for j in range(n) if i < j]
    def match_to_ij(m):
        if m > n*(n - 1)//2:
            raise Exception("m to ij out of range")
        return l[m]

    def ij_to_match(i,j):
        for m, ij in enumerate(l):
            if ij == (i,j):
                if m > n*(n - 1)//2:
                    raise Exception("ij to m out of range")
                return m

    # constraints
    model.one_match_a_week_per_team = ConstraintList()
    for i in range(n):
        for j in range(n):
            if i < j:
                model.one_match_a_week_per_team.add(sum(model.X[w, ij_to_match(i,j)] for w in model.W) == 1)
    # def home_away_rule(model, m):
    #     i,j = match_to_ij(m)
    #     return model.H[i]+model.H[j] == 1
    # model.home_away = Constraint(model.M, rule=home_away_rule)
    for m in range(n//2):
        model.H[m].fix(1)   # temporary

    def match_period_rule(model, i, j):
        if i < j:
            return sum(model.Q[p, i, j] for p in model.P) == 1
        return Constraint.Skip
    model.match_period = Constraint(model.I, model.J, rule=match_period_rule)

    # def match_period_i_rule(model, p, i):
    #     if i < n-2:
    #         return sum(model.Q[p, i, j] for j in model.J if i < j) <= 2
    #     return Constraint.Skip
    # def match_period_j_rule(model, p, j):
    #     if j > 0:
    #         return sum(model.Q[p, i, j] for i in model.I if i < j) <= 2
    #     return Constraint.Skip
    # model.match_period_i = Constraint(model.P, model.I, rule=match_period_i_rule)
    # model.match_period_j = Constraint(model.P, model.J, rule=match_period_j_rule)
    model.match_period_rule = ConstraintList()
    for p in model.P:
        for k in range(n):
            model.match_period_rule.add(sum(model.Q[p, k, j] for j in range(k,n)) + \
                                        sum(model.Q[p, i, k] for i in range(0,k)) <= 2)

    model.obj = Objective(expr=1, sense=minimize)

    solver = SolverFactory('glpk')
    result = solver.solve(model, tee=False, timelimit=300)

    solution = np.zeros((n-1, n//2, n, n))
    for w in model.W:
      for m in model.M:
          if model.X[w, m].value == 1:
              for p in model.P:
                  i,j = match_to_ij(m)
                  if model.Q[p, i, j].value == 1:
                      i, j = l[m]
                      if model.H[m].value == 1:
                          solution[w, p, i, j] = 1
                      else:
                          solution[w, p, j, i] = 1
    return result, solution

### 3 var solve optimization

In [6]:
def solve3opt(n, solver):
    start_cosntr = time.time()
    l = [(i,j) for i in range(n) for j in range(n) if i < j]
    ij_to_match = {(i, j): idx for idx, (i, j) in enumerate(l)}

    model = ConcreteModel()
    model.W = RangeSet(0, (n-1)-1)
    model.P = RangeSet(0, (n//2)-1)
    model.I = RangeSet(0, n-1)
    model.J = RangeSet(0, n-1)
    model.M = RangeSet(0, (n*(n - 1)//2)-1)
    model.WP = Set(initialize=[(w, p) for w in model.W for p in model.P])
    model.match_teams = Param(model.M, initialize=lambda model, m: l[m], within=Any)
    model.Y = Var(model.WP, model.M, domain=Binary)
    model.H = Var(model.M, domain=Binary)
    model.Home = Var(model.I, domain=Integers, bounds=(0, n-1))
    model.Away = Var(model.I, domain=Integers, bounds=(0, n-1))
    model.Z = Var(domain=Integers, bounds=(0, n-1))

    # necessary constraints
    model.max_one_match_per_week = ConstraintList()
    for w in model.W:
        for k in model.I:
            model.max_one_match_per_week.add(
                sum(
                    model.Y[(w, p), ij_to_match[(k, j)]]
                    for p in model.P for j in range(k + 1, n)
                ) +
                sum(
                    model.Y[(w, p), ij_to_match[(i, k)]]
                    for p in model.P for i in range(0, k)
                ) <= 1
            )

    def one_match_per_period_per_week_rule(model, w, p):
        return sum(model.Y[(w, p), m] for m in model.M) == 1
    model.one_match_per_period_per_week = Constraint(model.WP, rule=one_match_per_period_per_week_rule)

    def match_scheduled_once_rule(model, m):
        return sum(model.Y[wp, m] for wp in model.WP) == 1
    model.match_scheduled_once = Constraint(model.M, rule=match_scheduled_once_rule)

    model.max_team_match_period = ConstraintList()
    for p in model.P:
        for k in range(n):
            model.max_team_match_period.add(sum(model.Y[(w, p), ij_to_match[(k, j)]] for w in model.W for j in range(k+1,n)) + \
                                        sum(model.Y[(w, p), ij_to_match[(i, k)]] for w in model.W for i in range(0,k)) <= 2)

    # additional constraints for efficiency
    def tot_matches_rule(model):
        return sum(model.Y[wp,m] for wp in model.WP for m in model.M) == n*(n - 1)//2
    model.tot_matches = Constraint(rule=tot_matches_rule)

    # objective constraints
    def home_games_rule(model, i):
            return model.Home[i] == sum(model.H[m] for m in model.M if i == model.match_teams[m][0]) + \
                               sum(1 - model.H[m] for m in model.M if i == model.match_teams[m][1])
    model.home_games = Constraint(model.I, rule=home_games_rule)
    def away_games_rule(model, i):
            return model.Away[i] == sum(1 - model.H[m] for m in model.M if i == model.match_teams[m][0]) + \
                                 sum(model.H[m] for m in model.M if i == model.match_teams[m][1])
    model.away_games = Constraint(model.I, rule=away_games_rule)
    model.balance_max = Constraint(model.I, range(2), rule=lambda model, i, d:
        (model.Home[i] - model.Away[i] <= model.Z) if d == 0 else
        (model.Away[i] - model.Home[i] <= model.Z)
    )
    model.obj = Objective(expr=model.Z, sense=minimize)

    constr_time = time.time() - start_cosntr
    solver = SolverFactory(solver)
    solver.options['presolve'] = 1
    solver.options['mip_cuts'] = 'on'
    result = solver.solve(model, tee=False, timelimit=int(300-constr_time-1))  # tee=True for debug

    solution = np.zeros((n-1, n//2, n, n))
    for w in model.W:
        for p in model.P:
            for m in model.M:
                if model.Y[(w, p), m].value is not None and abs(model.Y[(w, p), m].value) > 1e-6:
                    i,j = model.match_teams[m]
                    if model.H[m].value == 1:
                        solution[w, p, i, j] = 1
                    else:
                        solution[w, p, j, i] = 1
    return result, solution

### circle matching

In [ ]:
def solve4(n, solver='cbc'):
    l = [(i,j) for i in range(n) for j in range(n) if i < j]
    ij_to_match = {(i, j): idx for idx, (i, j) in enumerate(l)}


    model = ConcreteModel()
    model.W = RangeSet(0, (n-1)-1)
    model.P = RangeSet(0, (n//2)-1)
    model.I = RangeSet(0, n-1)
    # model.J = RangeSet(0, n-1)
    model.M = RangeSet(0, (n*(n - 1)//2)-1)
    model.WP = Set(initialize=[(w, p) for w in model.W for p in model.P])
    model.match_teams = Param(model.M, initialize=lambda model, m: l[m], within=Any)
    model.Y = Var(model.WP, model.M, domain=Binary)
    model.H = Var(model.M, domain=Binary)
    model.Home = Var(model.I, domain=Integers, bounds=(0, n-1))
    model.Away = Var(model.I, domain=Integers, bounds=(0, n-1))
    model.Z = Var(domain=Integers, bounds=(0, n-1))

    # presolving
    presolved_M = np.zeros((n-1, n, n))
    pivot, circle = n, list(range(1,n))
    for w in range(1, n):
        presolved_M[w-1, circle[w-1]-1, pivot-1] = 1
        for k in range(1, n//2):
            i = circle[(w-1 + k) % (n-1)]-1
            j = circle[(w-1 - k) % (n-1)]-1
            if i < j:
                presolved_M[w-1, i, j] = 1
            else:
                presolved_M[w-1, j, i] = 1

    model.presolve_assignment = ConstraintList()
    for w in model.W:
        for p in model.P:
            for m in model.M:
                i,j = model.match_teams[m]
                model.presolve_assignment.add(model.Y[(w,p), m] <= presolved_M[w, i, j])

    # necessary constraints
    def one_match_per_period_per_week_rule(model, w, p):
        return sum(model.Y[(w, p), m] for m in model.M) == 1
    model.one_match_per_period_per_week = Constraint(model.WP, rule=one_match_per_period_per_week_rule)

    def match_scheduled_once_rule(model, m):
        return sum(model.Y[wp, m] for wp in model.WP) == 1
    model.match_scheduled_once = Constraint(model.M, rule=match_scheduled_once_rule)

    model.max_team_match_period = ConstraintList()
    for p in model.P:
        for k in range(n):
            model.max_team_match_period.add(sum(model.Y[(w, p), ij_to_match[(k, j)]] for w in model.W for j in range(k+1,n)) + \
                                        sum(model.Y[(w, p), ij_to_match[(i, k)]] for w in model.W for i in range(0,k)) <= 2)

    # additional constraints for efficiency
    def tot_matches_rule(model):
        return sum(model.Y[wp,m] for wp in model.WP for m in model.M) == n*(n - 1)//2
    model.tot_matches = Constraint(rule=tot_matches_rule)

    # objective constraints
    def home_games_rule(model, i):
        return model.Home[i] == sum(model.H[m] for m in model.M if i == model.match_teams[m][0]) + \
                               sum(1 - model.H[m] for m in model.M if i == model.match_teams[m][1])
    model.home_games = Constraint(model.I, rule=home_games_rule)
    def away_games_rule(model, i):
        return model.Away[i] == sum(1 - model.H[m] for m in model.M if i == model.match_teams[m][0]) + \
                                 sum(model.H[m] for m in model.M if i == model.match_teams[m][1])
    model.away_games = Constraint(model.I, rule=away_games_rule)
    model.balance_max = Constraint(model.I, range(2), rule=lambda model, i, d:
        (model.Home[i] - model.Away[i] <= model.Z) if d == 0 else
        (model.Away[i] - model.Home[i] <= model.Z)
    )
    model.obj = Objective(expr=model.Z, sense=minimize)

    solver = SolverFactory(solver)
    result = solver.solve(model, tee=False, timelimit=300)  # tee=True for debug

    solution = np.zeros((n-1, n//2, n, n))
    for w in model.W:
        for p in model.P:
            for m in model.M:
                if model.Y[(w, p), m].value is not None and abs(model.Y[(w, p), m].value) > 1e-6:
                    i,j = model.match_teams[m]
                    if model.H[m].value == 1:
                        solution[w, p, i, j] = 1
                    else:
                        solution[w, p, j, i] = 1
    return result, solution


In [ ]:
def circle_matchings(n):
    # standard “pivot + circle” 1-factorization
    pivot, circle = n, list(range(1,n))
    weeks = n-1
    m = {}
    for w in range(1, weeks+1):
        ms = [(pivot-1, circle[w-1]-1)]
        for k in range(1, n//2):
            i = circle[(w-1 + k) % (n-1)]-1
            j = circle[(w-1 - k) % (n-1)]-1
            ms.append((i,j))
        m[w-1] = ms
    return m

n = 4
matches_by_week = circle_matchings(n)
print(matches_by_week)

{0: [(3, 0), (1, 2)], 1: [(3, 1), (2, 0)], 2: [(3, 2), (0, 1)]}


## problem sol

In [ ]:
n = 12

# result, solution = solve(n)
# print(result)
# result, solution = solveSB(n)
# print(result)
# result, solution, result2, solution2 = solve1(n, ['glpk', 'cbc'])
# result, solution, result2, solution2 = solve2(n, ['glpk', 'cbc'])
result, solution = solve3opt(n, 'cbc')
# result, solution = solve4(n, 'cbc')

print(result)
# print(result2)

### Constraint verification


In [27]:
X = solution
# print(X)

s = np.sum(X, axis=(2,3))
print('solution matrix compressed for weeks/periods:\n', s)

m = np.sum(X, axis=(0,1))
# print('solution matrix compressed for matches:\n', m)

d = np.zeros_like(m)
for i in range(n):
    for j in range(n):
        if j <= i:
            continue
        d[i,j] = m[i,j]
print('diagonal compressed solution matrix for matches:\n', m+d.T-d)

# print('matches played in each period per team:')
max_p = 0
for i in range(n):
    # print(f"Team {i}:")
    for p in range(n//2):
        # print(f"  Periodo {p}: {np.sum(X[:, p, i, :])+np.sum(X[:, p, :, i])} partite")
        if np.sum(X[:, p, i, :])+np.sum(X[:, p, :, i]) > max_p:
            max_p = np.sum(X[:, p, i, :])+np.sum(X[:, p, :, i])
print('max matches played in one period: ', max_p)

max_i = 0
# print('imbalance per team:')
for i in range(n):
    # print(f"Team {i}: home {np.sum(m[:,i])}, away {np.sum(m[i,:])} => diff {abs(np.sum(m[:,i])-np.sum(m[i,:]))}")
    if abs(np.sum(m[:,i])-np.sum(m[i,:])) > max_i:
        max_i = abs(np.sum(m[:,i])-np.sum(m[i,:]))
print('max home/away imbalance: ', max_i)

solution matrix compressed for weeks/periods:
 [[1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]]
diagonal compressed solution matrix for matches:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.]]
max matches played in one period:  6.0
max home/away imbalance:  1.0


# MIP solution

In [80]:
def solve3opt(n, solver_name):
    start = time.time()

    assert n%2==0, "Number of teams must be even."
    Weeks   = list(range(n-1))
    Periods = list(range(n//2))
    Teams   = list(range(n))

    # Only i<j stored, but we’ll generate both (i,j) and (j,i) in constraints
    matches = [(i,j) for i in Teams for j in Teams if i<j]

    model = ConcreteModel()
    model.W = Set(initialize=Weeks)
    model.P = Set(initialize=Periods)
    model.Teams = Set(initialize=Teams)
    model.Matches = Set(initialize=matches, dimen=2)
    model.Y = Var(model.W, model.P, model.Matches, domain=Binary)
    ij_to_match = {(i, j): idx for idx, (i, j) in enumerate(model.Matches)}

    # 1) Each week+period hosts exactly one match
    def one_match_per_slot(m, w, p):
        return sum(m.Y[w,p,i,j] for (i,j) in m.Matches) == 1
    model.SlotConstraint = Constraint(model.W, model.P, rule=one_match_per_slot)

    # 2) Each unordered pair (i,j) plays exactly once
    def each_pair_once(m, i, j):
        return sum(m.Y[w,p,i,j] for w in m.W for p in m.P) == 1
    model.PairConstraint = Constraint(model.Matches, rule=each_pair_once)

    # 3) Each team plays exactly one game each week
    # def one_game_per_team_per_week(m, w, t):
    #     expr = []
    #     for (i,j) in m.Matches:
    #         if i==t or j==t:
    #             expr.append(m.Y[w, p, (i,j)] for p in m.P)
    #     return sum(_ for seq in expr for _ in seq) == 1
    # model.WeeklyTeamConstraint = Constraint(model.W, model.Teams, rule=one_game_per_team_per_week)
    # model.MaxOneMatchPerWeek = ConstraintList()
    # for w in model.W:
    #     for k in model.Teams:
    #         model.MaxOneMatchPerWeek.add(
    #             sum(model.Y[w, p, i, j]
    #                 for p in model.P
    #                 for (i, j) in model.Matches
    #                 if i == k or j == k) <= 1
    #         )

    # 4) At most two appearances in the same period
    def period_count_limit(m, p, t):
        # across all weeks, team t in period p at most twice
        expr = []
        for w in m.W:
            for (i,j) in m.Matches:
                if i==t or j==t:
                    expr.append(m.Y[w,p,(i,j)])
        return sum(expr) <= 2
    model.PeriodLimit = Constraint(model.P, model.Teams, rule=period_count_limit)

    # 5) Symmetry breaking: fix team 0’s matches
    def fix_team0(m, w, p, i, j):
        if i==0:
            # opponent is j, assign week=j-1, period=floor((j-1)/2)
            w0 = j-1
            p0 = (j-1)//2
            if (w,p)==(w0,p0):
                return m.Y[w,p,i,j] == 1
            else:
                return m.Y[w,p,i,j] == 0
        elif j==0:
            # match (i,0) is stored only as (0,i), so skip
            return Constraint.Skip
        else:
            return Constraint.Skip
    model.FixTeam0 = Constraint(model.W, model.P, model.Matches, rule=fix_team0)

    # Solver setup
    solver = SolverFactory(solver_name)
    # aggressive presolve & focus on feasible
    if solver_name.lower() == 'gurobi':
        solver.options['Presolve'] = 2
        solver.options['MIPFocus'] = 1
    elif solver_name.lower() == 'cplex':
        solver.options['preind'] = 1
        solver.options['mipfocus'] = 1

    constr_time = time.time() - start
    result = solver.solve(model, tee=False, timelimit=int(300-constr_time-1))

    # extract a human-readable schedule
    schedule = np.zeros((n-1, n//2, n, n))
    for w in Weeks:
        for p in Periods:
            for (i,j) in matches:
                if model.Y[w,p,i,j].value is not None and model.Y[w,p,i,j].value > 0.5:
                    schedule[w,p,i,j] = 1
    return result, schedule

In [81]:
import time
n = 12

start = time.time()
result, solution = solve3opt(n, 'cbc')
elapsed_time = time.time() - start

print(result)


Problem: 
- Name: unknown
  Lower bound: 1.0
  Upper bound: 1.0
  Number of objectives: 1
  Number of constraints: 176
  Number of variables: 3025
  Number of binary variables: 4356
  Number of integer variables: 4356
  Number of nonzeros: 0
  Sense: minimize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.42
  Wallclock time: 0.43
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.4668233394622803
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [73]:
X = solution
# print(X)

s = np.sum(X, axis=(2,3))
print('solution matrix compressed for weeks/periods:\n', s)

m = np.sum(X, axis=(0,1))
# print('solution matrix compressed for matches:\n', m)

d = np.zeros_like(m)
for i in range(n):
    for j in range(n):
        if j <= i:
            continue
        d[i,j] = m[i,j]
print('diagonal compressed solution matrix for matches:\n', m+d.T-d)

# print('matches played in each period per team:')
max_p = 0
for i in range(n):
    # print(f"Team {i}:")
    for p in range(n//2):
        # print(f"  Periodo {p}: {np.sum(X[:, p, i, :])+np.sum(X[:, p, :, i])} partite")
        if np.sum(X[:, p, i, :])+np.sum(X[:, p, :, i]) > max_p:
            max_p = np.sum(X[:, p, i, :])+np.sum(X[:, p, :, i])
print('max matches played in one period: ', max_p)

max_i = 0
# print('imbalance per team:')
for i in range(n):
    # print(f"Team {i}: home {np.sum(m[:,i])}, away {np.sum(m[i,:])} => diff {abs(np.sum(m[:,i])-np.sum(m[i,:]))}")
    if abs(np.sum(m[:,i])-np.sum(m[i,:])) > max_i:
        max_i = abs(np.sum(m[:,i])-np.sum(m[i,:]))
print('max home/away imbalance: ', max_i)

solution matrix compressed for weeks/periods:
 [[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
diagonal compressed solution matrix for matches:
 [[0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 0.]]
max matches played in one period:  2.0
max home/away imbalance:  7.0


In [74]:
l = [(i,j) for i in range(n) for j in range(n) if i < j]
formatted_sol = []
for p in range(n//2):
    row = []
    for w in range(n-1):
        for i in range(n):
            for j in range(n):
                if solution[w,p,i,j] == 1:
                    row.append([i+1, j+1])
    formatted_sol.append(row)

output = {
    "sol": formatted_sol,
    "time": elapsed_time,
    "optimal": formatted_sol,
    "obj": result.Problem.Upper_bound,
}

## json formatting

In [75]:
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump({'MIP':output}, f, ensure_ascii=False, indent=4)

## solution checker

In [76]:
def get_elements(solution):
    periods = [s for s in solution]
    matches = [m for s in periods for m in s]
    teams = [t for m in matches for t in m]

    return periods, matches, teams


def get_weeks(periods, n):
    return [[p[i] for p in periods] for i in range(n-1)]


def fatal_errors(solution, obj, time, optimal, teams):
    fatal_errors = []

    if len(solution) == 0 and (not (time == 300 and not optimal) and not (time == 0 and optimal) and obj!='None'):
        fatal_errors.append('The solution cannot be empty!!!')
        return fatal_errors

    if type(solution) != list:
        fatal_errors.append('The solution should be a list!!!')
        return fatal_errors

    if len(solution) > 0:
        if any([t not in set(teams) for t in range(1,n+1)]):
            fatal_errors.append(f'Missing team in the solution or team out of range!!!')

        if n%2 != 0:
            fatal_errors.append(f'"n" should be even!!!')

        if len(solution) != n//2:
            fatal_errors.append(f'the number of periods is not compliant!!!')

        if any([len(s) != n - 1 for s in solution]):
            fatal_errors.append(f'the number of weeks is not compliant!!!')

        if time > 300:
            fatal_errors.append(f'The running time exceeds the timeout!!!')

    return fatal_errors


def check_solution(solution: list, obj, time, optimal):

    periods, solution_matches, teams = get_elements(solution)

    errors = fatal_errors(solution, obj, time, optimal, teams)

    if len(errors) == 0 and len(solution) > 0:

        n = max(teams)

        teams_matches = combinations(set(teams),2)

        # every team plays with every other teams only once
        if any([solution_matches.count([h,a]) + solution_matches.count([a,h]) > 1 for h,a in teams_matches]):
            errors.append('There are duplicated matches')

        # each team cannot play against itself
        if any([h==a for h,a in solution_matches]):
            errors.append('There are self-playing teams')

        weeks = get_weeks(periods, n)

        # every team plays once a week
        teams_per_week = [[j for i in w for j in i] for w in weeks]
        if any([len(tw) != len(set(tw)) for tw in teams_per_week]):
            errors.append('Some teams play multiple times in a week')

        teams_per_period = [[j for i in p for j in i] for p in periods]

        # every team plays at most twice during the period
        if any([any([tp.count(elem) > 2 for elem in tp]) for tp in teams_per_period]):
            errors.append('Some teams play more than twice in the period')

    return 'Valid solution' if len(errors) == 0 else errors


def load_json(path):
    try:
        with open(path, 'r') as f:
            return json.load(f)
    except Exception as e:
        print(f"Error reading {path}: {e}")
        sys.exit(1)


# if __name__ == '__main__':

# parser = argparse.ArgumentParser(description="Check the validity of a STS solution JSON file.")
# parser.add_argument("json_file_directory", help="Path to the directory containing .json solution files")
# args = parser.parse_args()
# directory = args.json_file_directory
directory = './'
for f in filter(lambda x: x.endswith('.json'), os.listdir(directory)):
    json_data = load_json(f'{directory}/{f}')
    print(f'File: {f}\n')
    for approach, result in json_data.items():
        sol = result.get("sol")
        time = result.get("time")
        opt = result.get("optimal")
        obj = result.get("obj")
        message = check_solution(sol, obj, time, opt)
        status = "VALID" if type(message) == str else "INVALID"
        message_str = '\n\t  '.join(message)
        print(f"  Approach: {approach}\n    Status: {status}\n    Reason: {message if status == 'VALID' else message_str}\n")

File: data.json

  Approach: MIP
    Status: VALID
    Reason: Valid solution

